Xóa những ảnh ko có label tương đương và ngược lại

In [26]:
import os

def sync_images_and_labels(image_dir, label_dir, image_extensions=(".jpg", ".png", ".jpeg"), label_extension=".txt"):
    """
    Đồng bộ thư mục ảnh và nhãn:
    - Xóa tệp ảnh không có nhãn tương ứng.
    - Xóa tệp nhãn không có ảnh tương ứng.

    Args:
        image_dir (str): Đường dẫn đến thư mục ảnh.
        label_dir (str): Đường dẫn đến thư mục nhãn.
        image_extensions (tuple): Các phần mở rộng hợp lệ của ảnh (mặc định là .jpg, .png, .jpeg).
        label_extension (str): Phần mở rộng của tệp nhãn (mặc định là .txt).
    """
    # Lấy danh sách tên tệp (không tính phần mở rộng) trong thư mục ảnh và nhãn
    image_files = {os.path.splitext(filename)[0] for filename in os.listdir(image_dir) if filename.lower().endswith(image_extensions)}
    label_files = {os.path.splitext(filename)[0] for filename in os.listdir(label_dir) if filename.endswith(label_extension)}
    
    # Xóa tệp ảnh không có nhãn tương ứng
    for image_file in os.listdir(image_dir):
        if image_file.lower().endswith(image_extensions):
            image_name = os.path.splitext(image_file)[0]
            if image_name not in label_files:
                image_path = os.path.join(image_dir, image_file)
                os.remove(image_path)
                print(f"Đã xóa ảnh: {image_path}")
    
    # Xóa tệp nhãn không có ảnh tương ứng
    for label_file in os.listdir(label_dir):
        if label_file.endswith(label_extension):
            label_name = os.path.splitext(label_file)[0]
            if label_name not in image_files:
                label_path = os.path.join(label_dir, label_file)
                os.remove(label_path)
                print(f"Đã xóa nhãn: {label_path}")

# Ví dụ sử dụng
image_dir = r"D:\BKAI\main_datasets\nighttime\nighttime_fold_2\val\image"
label_dir = r"D:\BKAI\main_datasets\nighttime\nighttime_fold_2\val\label"

sync_images_and_labels(image_dir, label_dir)


Đã xóa ảnh: D:\BKAI\main_datasets\nighttime\nighttime_fold_2\val\image\cam_05_00016_jpg.rf.0cf018d2dfbe500503a57ba6bed53d6d.jpg
Đã xóa ảnh: D:\BKAI\main_datasets\nighttime\nighttime_fold_2\val\image\cam_05_00020_jpg.rf.02b82a768be0afcddc2c15e1bdd24e9b.jpg
Đã xóa ảnh: D:\BKAI\main_datasets\nighttime\nighttime_fold_2\val\image\cam_05_00020_jpg.rf.28bc22459526b9e2efcfcbed11ffa58b.jpg
Đã xóa ảnh: D:\BKAI\main_datasets\nighttime\nighttime_fold_2\val\image\cam_05_00041_jpg.rf.bb2c4873d1eda33468e671adad0e9498.jpg
Đã xóa ảnh: D:\BKAI\main_datasets\nighttime\nighttime_fold_2\val\image\cam_05_00041_jpg.rf.ce0949e2cb6429ce4828b9fce2caab93.jpg
Đã xóa ảnh: D:\BKAI\main_datasets\nighttime\nighttime_fold_2\val\image\cam_05_00042_jpg.rf.968b6d4efa450a2f4cd7820ce26587c3.jpg
Đã xóa ảnh: D:\BKAI\main_datasets\nighttime\nighttime_fold_2\val\image\cam_05_00042_jpg.rf.d2506b5aa92193043b9e13dd19eb8e30.jpg
Đã xóa ảnh: D:\BKAI\main_datasets\nighttime\nighttime_fold_2\val\image\cam_05_00047_jpg.rf.fba327661034c

In [27]:
import os
import json
import cv2

def coco2json(image_dir, label_dir, json_file):
    image_filenames = os.listdir(image_dir)

    dataset = {
        "images": [],
        "annotations": [],
        "categories": [
            {"id": 0, "name": "motorbike"},
            {"id": 1, "name": "car"},
            {"id": 2, "name": "coach"},
            {"id": 3, "name": "truck"},
        ]
    }
    annotation_id = 0

    # cam_01_00001.jpg
    for image_filename in image_filenames:
        
        if '.json' in image_filename:
            continue

        # The id should be cam_01_00001
        image_id = image_filename.split('.jpg')[0]
        image = cv2.imread(os.path.join(image_dir, image_filename))
        height, width, _ = image.shape

        dataset['images'].append({
            'id': image_id,
            'file_name': image_filename,
            'width': width,
            'height': height,
        })

        label_filename = image_filename.replace('.jpg', '.txt')
        with open(os.path.join(label_dir, label_filename)) as f:
            lines = f.readlines()
            for line in lines:
                # YOLO format: category_id, x_center, y_center, w, h
                # Normalized by image width and height
                # Need to convert to COCO format: x, y, w, h (no normalized)
                category_id, x_center, y_center, w, h = line.split(' ')
                x_center = float(x_center)
                y_center = float(y_center)
                w = float(w)
                h = float(h)

                x = (x_center - w / 2) * width
                y = (y_center - h / 2) * height
                w = w * width
                h = h * height

                dataset['annotations'].append({
                    'id': annotation_id,
                    'image_id': image_id,
                    'category_id': int(category_id),
                    'bbox': [x, y, w, h],
                    'area': w * h,
                    'iscrowd': 0,
                })

                annotation_id += 1

    with open(json_file, 'w') as f:
        json.dump(dataset, f, indent=4)

coco2json(r"D:\BKAI\main_datasets\nighttime\nighttime_fold_2\val\image", r"D:\BKAI\main_datasets\nighttime\nighttime_fold_2\val\label", r"D:\BKAI\main_datasets\nighttime\nighttime_fold_2\val\image\annotations.json")

Sắp xếp tên các file .txt theo số lượng tăng dần số class